<a href="https://colab.research.google.com/github/Cuong22MSE23093/CapstoneProject/blob/main/Electric_theft_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'electricity-theft-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2814103%2F4854639%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240720%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240720T115907Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7c1b518f0e75c724d5d470541c306cf7fde8c3f1024bd2e850c47cfba4e3190b9af7218a10ade1557cff410c30216021a90d751613c3627086c60e417236074f5a5fe3b6c339b9ce296ee0989af0df6c2733bd8060bc01396bfef672b5f7cc2d9dc85c9a45fa1f7db8d72bca82dcd5d441450c39ececb447d28e2674f878ce5919a1d0a805262e3697c45fb84aa560549dca3b56c8aea462ac83951a1db377a97186bc9d1c6164ecbe8a18bbb94aea265ebd80359829fbc861d7260d922f16f723cd523b49b60c5eedf3001f9e03061205446b4482db24875d3827e92f65e3124ac48f8ff828a84b65c74ca2ef20ac4db2f34f40cf1aedc789b6736fe0efdbb7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

from pykalman import KalmanFilter
from scipy.fft import fft, ifft
from datetime import datetime

In [ ]:
theft_data_1 = pd.read_csv('/kaggle/input/electricity-theft-detection/data.csv')

In [ ]:
theft_data_1.shape

In [ ]:
theft_data_1.head()

In [ ]:
theft_data_1.columns

In [ ]:
theft_data = theft_data_1.set_index('CONS_NO').copy()

In [ ]:
date_feature = theft_data.columns.drop(['FLAG']).copy()

In [ ]:
date_feature

In [ ]:
feature_df = theft_data[date_feature]
feature_df

In [ ]:
feature_df_T = feature_df.T
feature_df_T.index.names = ['Date'] #Change the name of index
TARGET = theft_data['FLAG']
TARGET_T = TARGET.T

In [ ]:
feature_df_T.head()

In [ ]:
feature_df_T.index = pd.to_datetime(feature_df_T.index, infer_datetime_format=True)

In [ ]:
feature_df_T.index.name = 'Date'

In [ ]:
number_of_feature = len(feature_df_T.columns)
print(number_of_feature)

In [ ]:

Residental_list = []
for i in range(number_of_feature):
    Residental_list.append('Resident' + ' ' + str(i))


feature_df_T.columns = Residental_list

In [ ]:
feature_df_T_2014 = feature_df_T['2014-01-01':'2014-12-31'].copy()
index_month = feature_df_T_2014.index.month
mean_by_month= feature_df_T_2014.groupby(index_month).mean()

In [ ]:
mean_by_month.head(5)

In [ ]:
mean_sample = mean_by_month.loc[:,['Resident 6', 'Resident 7', 'Resident 9']]

plt.style.use('ggplot')
ax1 = mean_sample.plot(colormap = 'Dark2', figsize = (10, 5), linewidth = 2)

ax1.set_xlabel('Resident', fontsize = 10)
ax1.set_ylabel('Mean Electricity by month')
ax1.set_title('Mean Electricity by month of 3 sample residental')
plt.show()

 # Missing value

In [ ]:
print(len(feature_df_T))

In [ ]:
# Dop all the columns with more than 70% missing value
threshold_percent = 70
total_missing = feature_df_T.isnull().sum()

percent_missing = (total_missing*100/len(feature_df_T))
drop_cols = list(percent_missing[total_missing > threshold_percent].index)
feature_df_T = feature_df_T.drop(drop_cols, axis=1).copy()

feature_df_T.head(5)

In [ ]:
feature_df_T.isnull().sum().plot(figsize = (6,8), color = 'lightgreen')
plt.show()

In [ ]:
feature_df_T['Resident 6'].plot()
plt.show()

In [ ]:
fig, ax = plt.subplots(2, figsize = (12, 5))
plot_acf(feature_df_T['Resident 7'].dropna(), lags = 60, ax=ax[0])
plot_pacf(feature_df_T['Resident 7'].dropna(), lags = 60, ax=ax[1])
plt.show()

In [ ]:
feature_df_T.head(5)


In [ ]:
feature_df_T.isnull().sum()

In [ ]:
daterange = pd.date_range(start=feature_df_T.index.min(), end=feature_df_T.index.max(), freq='D')
feature_df_T_calenderized = pd.DataFrame(index=daterange, columns=feature_df_T.columns)
feature_df_T_calenderized.update(feature_df_T)

In [ ]:
feature_df_T_calenderized.head(5)

In [ ]:
feature_df_T_calenderized.isnull().sum()

In [ ]:
feature_df_T.isnull().sum()

# EDA

In [ ]:
start_date = feature_df_T.index.min()
end_date = feature_df_T.index.max()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
expected_days = len(date_range)
actual_days = len(feature_df_T_calenderized)
print(f"Expected days: {expected_days}")
print(f"Actual days: {actual_days}")

In [ ]:
# If we want to use linear model (LDA or SVM), use feature_df_T_diff data
feature_df_T_diff = feature_df_T_calenderized.diff()
feature_df_T_diff = feature_df_T_diff['2014-01-02':].copy()


In [ ]:
feature_df_T_diff.isnull().sum()

In [ ]:
feature_df_T_diff['Resident 6'].plot(figsize = (12,7))
plt.show()

In [ ]:
#Removing outlier
'''
for col in feature_df_T_diff.columns:
    col_diff = feature_df_T_diff[col]
    q1 = np.percentile(col_diff, 25)
    q3 = np.percentile(col_diff, 75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr
    lower_bound = q1 - 1.5 * iqr
    outliers = np.where((col_diff > upper_bound) | (col_diff < lower_bound))[0]
    # Replace outliers with NaN values
    col_diff.iloc[outliers] = np.nan
'''


In [ ]:
feature_df_T_diff = feature_df_T_diff.astype('float64')

In [ ]:
feature_df_T_diff.dtypes

In [ ]:
missing_indices = feature_df_T_diff.isna()

In [ ]:
feature_df_T_diff.isnull().sum()

In [ ]:
print(feature_df_T_diff.index)

In [ ]:
feature_df_T_diff.index.freq

In [ ]:
missing_indices.head()

In [ ]:
feature_df_T_calenderized.isnull().sum()

In [ ]:
feature_df_T_calenderized["Resident 6"].plot(figsize = (15,7))
plt.show()

In [ ]:
feature_df_T = feature_df_T_calenderized.copy()

In [ ]:
feature_df_T = feature_df_T.copy().fillna(0)
n_cols = feature_df_T.shape[1]
n_cols

In [ ]:
kf = KalmanFilter(n_dim_obs=n_cols, n_dim_state=n_cols,
                  em_vars=['transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance'])

filtered_state_means, filtered_state_covariances = kf.filter(feature_df_T.values)
filtered_data = pd.DataFrame(filtered_state_means, index=feature_df_T.index, columns=feature_df_T.columns)

In [ ]:
'''
feature_df_T = feature_df_T_calenderized.copy()
freq_data = np.zeros_like(feature_df_T)
for i in range(feature_df_T.shape[1]):
    freq_data[:, i] = fft(feature_df_T.iloc[:, i].values)
'''

In [ ]:
plt.plot(freq_data)
plt.show()

In [ ]:
feature_df_T.head(5)

In [ ]:
feature_df_T.shape

In [ ]:
feature_df_T_calenderized.isnull().sum()

In [ ]:
feature_df_T.isnull().sum()

In [ ]:
'''max_freq = feature_df_T.shape[0] // 2 # Ignore negative frequencies due to symmetry
threshold = 0.1 # Set threshold value for identifying noise frequencies
noise_freqs = np.where(np.abs(freq_data[:max_freq]) > threshold)[0]

# Filter out noise frequencies for each column separately
freq_data[:, noise_freqs] = 0

# Apply the inverse Fourier Transform to obtain denoised time series
denoised_data = np.zeros_like(feature_df_T)
for i in range(feature_df_T.shape[1]):
    denoised_data[:, i] = np.real(ifft(freq_data[:, i]))

# Add the denoised time series data back to the data frame
feature_df_T = denoised_data.copy()'''


In [ ]:
decoArtistmposition = seasonal_decompose(feature_df_T, model='additive', period = 730)
seasonal = decomposition.seasonal

In [ ]:
date_feature

In [ ]:
X_theft = theft_data_1.drop(['FLAG'], axis=1)
y_theft = theft_data_1['FLAG']

In [ ]:
date_feature

In [ ]:
X_theft.loc[len(X_theft.index)] =  date_feature

In [ ]:
X_theft